In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import json
import re
from time import time

from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# ONE VS REST ON TOOLS 

In [4]:
# Load Data
df_tools=pd.read_csv('TOOLS_COMPLETELY_CLEAN_FULLY_MERGED_RDY_FOR_MDLLING')

In [5]:
df_tools.shape

(2267345, 10)

In [6]:
df_tools.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE
0,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,AL19QO4XLBQPU,5.0,True,"returned, decided against this product",2018-01-28
1,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A1I7CVB7X3T81E,5.0,True,Awesome heater for the electrical requirements...,2017-11-30
2,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A1AQXO4P5U674E,5.0,True,Keeps the mist of your wood trim and on you. B...,2017-09-12
3,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,AIRV678P7C4NK,4.0,True,"So far I hooked it up and tested it , filled a...",2017-07-19
4,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A22I5QDNTNECDW,1.0,True,"i installed this 10 months ago, instructions w...",2017-05-25


In [7]:
df_tools.drop_duplicates(subset=None,keep='first',inplace=True)

In [8]:
df_tools.shape

(1817775, 10)

In [9]:
sentiment=[]
for i in df_tools['rating']:
    if (i>3):
        sentiment.append('Positive')
    elif(i<3):
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')

In [10]:
df_tools['ReviewSentiment']=sentiment

In [11]:
df_tools.shape

(1817775, 11)

In [12]:
df_tools.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE,ReviewSentiment
0,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,AL19QO4XLBQPU,5.0,True,"returned, decided against this product",2018-01-28,Positive
1,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A1I7CVB7X3T81E,5.0,True,Awesome heater for the electrical requirements...,2017-11-30,Positive
2,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A1AQXO4P5U674E,5.0,True,Keeps the mist of your wood trim and on you. B...,2017-09-12,Positive
3,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,AIRV678P7C4NK,4.0,True,"So far I hooked it up and tested it , filled a...",2017-07-19,Positive
4,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A22I5QDNTNECDW,1.0,True,"i installed this 10 months ago, instructions w...",2017-05-25,Negative


In [13]:
Text_Tools  = df_tools[["reviewText","ReviewSentiment"]]

In [14]:
Text_Tools.head()

,reviewText,ReviewSentiment
0,"returned, decided against this product",Positive
1,Awesome heater for the electrical requirements...,Positive
2,Keeps the mist of your wood trim and on you. B...,Positive
3,"So far I hooked it up and tested it , filled a...",Positive
4,"i installed this 10 months ago, instructions w...",Negative


# text prerpocessing 

In [15]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

print(len(stopwords))
#print(stopwords)

326


In [16]:
Text_Tools.head()

,reviewText,ReviewSentiment
0,"returned, decided against this product",Positive
1,Awesome heater for the electrical requirements...,Positive
2,Keeps the mist of your wood trim and on you. B...,Positive
3,"So far I hooked it up and tested it , filled a...",Positive
4,"i installed this 10 months ago, instructions w...",Negative


In [17]:
def clean_text(text):
    # Split the text into individual words
    words = text.split()
    
    # Remove any punctuation or non-alphabetic characters
    words = [word for word in words if word.isalpha()]
    
    # Convert all words to lowercase
    words = [word.lower() for word in words]
    
    # Remove any stop words
    stop_words = stopwords=('english')
    words = [word for word in words if word not in stop_words]
    
    # Join the words back into a single string
    return " ".join(words)


# Clean the "text" column using the clean_text() function
Text_Tools["reviewText"] = Text_Tools["reviewText"].apply(clean_text)

# Print the cleaned data
print(Text_Tools)

                                                reviewText ReviewSentiment
0                             decided against this product        Positive
1        awesome heater for the electrical makes an awe...        Positive
2         keeps the mist of your wood trim and on bendable        Positive
3        so far hooked it up and tested it filled a fiv...        Positive
4        installed this months instructions were easy a...        Negative
...                                                    ...             ...
2267340  the tool a handy drain especially for clearing...        Positive
2267341  this thing bathroom sink was so friend and dra...        Positive
2267342  purchased this exact product from a local box ...        Positive
2267343  have had one of these in the past and was curi...        Positive
2267344                    product works as thank you very        Positive

[1817775 rows x 2 columns]


In [18]:
Text_Tools["reviewText"]=Text_Tools["reviewText"].apply(lambda x:' '.join([word for word in x.split() if word not in stopwords]))

In [19]:
# checking for first 100 records

In [20]:
c = 0 
for i in Text_Tools["reviewText"]:
    print(i)
    c+=1
    if c==100:
        break

decided product
awesome heater electrical makes awesome preheater talnkless system
keeps mist wood trim bendable
far hooked tested filled gallon bucket hot perfect temp flow valve came broke tried tighten stop hooked directly sharkbite pex work fine hooked poll amp wait till rest bathroom finished better
installed months instructions easy worked water heater stuck tried different modes reset circuit heat water update warranty requires licensed useless service renders possible difficulty monetary gain company disturbs actually dropped heater fixed company named changed supposed heating element lasts years broken tried pick difficult pick possible worst try working voip callback system bad sound baby crying background gives away pretty product like kind repackaged alibaba celsius reading obviously usa jump hoops pick repaired left figured save problems road want deal bad service bought water installed require contractor install warranty new water heater readings dial adjusting inlet manu

In [21]:
print(stopwords)

{'side', 'me', 'all', 'when', '‘re', 'their', 'becomes', 'an', 'many', 'n‘t', 'anyway', 'unless', 'indeed', 'we', 'yet', "'re", 'for', '’d', 'they', 'or', 'several', 'hereupon', 'though', 'ca', 'every', 'up', 'again', 'get', 'whereby', 'nevertheless', 'everywhere', 'nowhere', 'therein', 'enough', 'such', 'just', 'my', 'third', "'ve", 'has', 'with', 'off', 'beforehand', 'its', 'towards', 'take', 'always', 'whence', 'are', 'anything', 'nine', 'really', 'against', 'last', 'latter', 'seeming', 'toward', 'hereby', 'two', 'why', 'more', '‘ll', 'something', 'everything', 'himself', 'on', 'most', 'along', 'and', 'does', 'down', 'whenever', 'done', 'otherwise', 'thereupon', 'during', 'also', 'throughout', 'meanwhile', '‘m', 'how', 'namely', 'yourself', 'see', 'therefore', 'another', 'perhaps', 'either', 'ten', 'below', 'both', 'next', 'hundred', 'serious', 'doing', 'others', 'upon', 'have', 'what', 'name', 'never', '‘ve', 'became', 'quite', 'keep', "n't", 'go', 'give', 'your', 'make', 'herein',

In [22]:
Text_Tools.head()

,reviewText,ReviewSentiment
0,decided product,Positive
1,awesome heater electrical makes awesome prehea...,Positive
2,keeps mist wood trim bendable,Positive
3,far hooked tested filled gallon bucket hot per...,Positive
4,installed months instructions easy worked wate...,Negative


In [23]:
# train test split 

In [24]:
X=[]
y=[]
for i in Text_Tools['reviewText']:
    X.append(i)
for j in Text_Tools['ReviewSentiment']:
    y.append(j)  

In [25]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=9)

In [26]:
len(X_train)

1454220

In [27]:
len(X_test)

363555

###### Converting text data to vector form using TfidfVectorizer()

In [28]:
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [29]:
print("X_train",X_train.shape)
print("X_test",X_test.shape)


X_train (1454220, 109944)
X_test (363555, 109944)


###### One vs rest strategy with Logistic Regression

In [30]:
lr = LogisticRegression(max_iter=200)
ovr = OneVsRestClassifier(lr)

In [31]:
# fit the model on the training data
ovr.fit(X_train, y_train)
y_pred = ovr.predict(X_test)

In [32]:
df_compare1=pd.DataFrame()
df_compare1['Actual']=y_test
df_compare1['Predicted']=y_pred

In [33]:
df_compare1

,Actual,Predicted
0,Positive,Positive
1,Positive,Positive
2,Positive,Positive
3,Positive,Positive
4,Positive,Positive
...,...,...
363550,Positive,Positive
363551,Positive,Positive
363552,Positive,Positive
363553,Neutral,Positive


## classification report 

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.67      0.45      0.54     29993
     Neutral       0.44      0.04      0.07     23977
    Positive       0.89      0.99      0.94    309585

    accuracy                           0.88    363555
   macro avg       0.67      0.49      0.52    363555
weighted avg       0.85      0.88      0.85    363555



### confusion matrix 

In [35]:
confusion_matrix(y_test,y_pred)

array([[ 13382,    474,  16137],
       [  3014,    937,  20026],
       [  3466,    729, 305390]], dtype=int64)

# evaluation matrix 

In [36]:
print('accuracy:', accuracy_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred, average='weighted'))
print('precision:', precision_score(y_test, y_pred,average='weighted'))

accuracy: 0.8793965149702246
recall: 0.8793965149702246
f1-score: 0.8477905491603605
precision: 0.845849467477149


In [37]:
TF_IDF_ACC = round(100*accuracy_score(y_test, y_pred),2)
TF_IDF_PREC =  round(100*precision_score(y_test, y_pred,average='weighted'),2)
TF_IDF_RE=  round(100*recall_score(y_test, y_pred, average='weighted'),2)
TF_IDF_F1 =  round(100*f1_score(y_test, y_pred, average='weighted'),2)

# tfidf with hyperparametre 

In [38]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=9)

In [39]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True,
                          max_df=0.5,
                          min_df=1,
                          use_idf=True,smooth_idf=True)
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [40]:
print("X_train",X_train.shape)
print("X_test",X_test.shape)


X_train (1454220, 109944)
X_test (363555, 109944)


# model training 

In [41]:
# fit the model on the training data
ovr.fit(X_train, y_train)
y_pred = ovr.predict(X_test)

In [42]:
df_compare1=pd.DataFrame()
df_compare1['Actual']=y_test
df_compare1['Predicted']=y_pred

In [43]:
df_compare1

,Actual,Predicted
0,Positive,Positive
1,Positive,Positive
2,Positive,Positive
3,Positive,Positive
4,Positive,Positive
...,...,...
363550,Positive,Positive
363551,Positive,Positive
363552,Positive,Positive
363553,Neutral,Positive


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.67      0.45      0.54     29993
     Neutral       0.44      0.04      0.07     23977
    Positive       0.89      0.99      0.94    309585

    accuracy                           0.88    363555
   macro avg       0.67      0.49      0.52    363555
weighted avg       0.85      0.88      0.85    363555



In [45]:
confusion_matrix(y_test,y_pred)

array([[ 13594,    478,  15921],
       [  3113,    949,  19915],
       [  3563,    738, 305284]], dtype=int64)

In [46]:
print('accuracy:', accuracy_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred, average='weighted'))
print('precision:', precision_score(y_test, y_pred,average='weighted'))

accuracy: 0.879721087593349
recall: 0.879721087593349
f1-score: 0.8484370916319444
precision: 0.8463274617454206


In [47]:
TFIDF_HYPER_ACC =  round(accuracy_score(y_test, y_pred),2)*100
TFIDF_HYPER_PREC = round(precision_score(y_test, y_pred,average='weighted'),2)*100
TFIDF_HYPER_RE = round(recall_score(y_test, y_pred, average='weighted'),2)*100
TFIDF_HYPER_F1 = round(f1_score(y_test, y_pred, average='weighted'),2)*100

 # bag of words ( uni gram ) 

In [48]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=9)

In [49]:
# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer()
X_train=cv.fit_transform(X_train)


print(X_train.shape)



X_test=cv.transform(X_test)

print(X_test.shape)

(1454220, 109944)
(363555, 109944)


In [50]:
print("X_train",X_train.shape)
print("X_test",X_test.shape)


X_train (1454220, 109944)
X_test (363555, 109944)


# model training 

In [51]:
# fit the model on the training data
ovr.fit(X_train, y_train)
y_pred = ovr.predict(X_test)

In [52]:
df_compare1=pd.DataFrame()
df_compare1['Actual']=y_test
df_compare1['Predicted']=y_pred

In [53]:
df_compare1

,Actual,Predicted
0,Positive,Positive
1,Positive,Positive
2,Positive,Positive
3,Positive,Positive
4,Positive,Positive
...,...,...
363550,Positive,Positive
363551,Positive,Positive
363552,Positive,Positive
363553,Neutral,Positive


In [54]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.66      0.40      0.50     29993
     Neutral       0.37      0.06      0.10     23977
    Positive       0.89      0.98      0.94    309585

    accuracy                           0.87    363555
   macro avg       0.64      0.48      0.51    363555
weighted avg       0.84      0.87      0.84    363555



In [55]:
confusion_matrix(y_test,y_pred)

array([[ 12100,    890,  17003],
       [  2758,   1384,  19835],
       [  3552,   1469, 304564]], dtype=int64)

In [56]:
print('accuracy:', accuracy_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred, average='weighted'))
print('precision:', precision_score(y_test, y_pred,average='weighted'))

accuracy: 0.8748277427074308
recall: 0.8748277427074308
f1-score: 0.8446266459930104
precision: 0.8382740618173977


In [57]:
BOW_UNI_ACC =  round(accuracy_score(y_test, y_pred),2)*100
BOW_UNI_PREC = round(precision_score(y_test, y_pred,average='weighted'),2)*100
BOW_UNI_RE = round(recall_score(y_test, y_pred, average='weighted'),2)*100
BOW_UNI_F1 = round(f1_score(y_test, y_pred, average='weighted'),2)*100

# n_gram  ( bi gram ) 

In [58]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=9)

In [59]:
# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer(ngram_range=(2,2))
X_train=cv.fit_transform(X_train)


print(X_train.shape)



X_test=cv.transform(X_test)

print(X_test.shape)

(1454220, 6109630)
(363555, 6109630)


# model training 

In [60]:
# fit the model on the training data
ovr.fit(X_train, y_train)
y_pred = ovr.predict(X_test)

In [61]:
df_compare1=pd.DataFrame()
df_compare1['Actual']=y_test
df_compare1['Predicted']=y_pred

In [62]:
df_compare1

,Actual,Predicted
0,Positive,Positive
1,Positive,Positive
2,Positive,Positive
3,Positive,Positive
4,Positive,Positive
...,...,...
363550,Positive,Positive
363551,Positive,Positive
363552,Positive,Positive
363553,Neutral,Positive


In [63]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.66      0.33      0.44     29993
     Neutral       0.41      0.09      0.14     23977
    Positive       0.89      0.98      0.93    309585

    accuracy                           0.87    363555
   macro avg       0.65      0.47      0.50    363555
weighted avg       0.84      0.87      0.84    363555



In [64]:
confusion_matrix(y_test,y_pred)

array([[  9838,   1114,  19041],
       [  2174,   2054,  19749],
       [  2824,   1829, 304932]], dtype=int64)

In [65]:
print('accuracy:', accuracy_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred, average='weighted'))
print('precision:', precision_score(y_test, y_pred,average='weighted'))

accuracy: 0.8714609893963774
recall: 0.8714609893963774
f1-score: 0.8404844275686301
precision: 0.8372652899511801


In [66]:
BOW_BI_ACC =  round(accuracy_score(y_test, y_pred),2)*100
BOW_BI_PREC = round(precision_score(y_test, y_pred,average='weighted'),2)*100
BOW_BI_RE = round(recall_score(y_test, y_pred, average='weighted'),2)*100
BOW_BI_F1 = round(f1_score(y_test, y_pred, average='weighted'),2)*100

In [67]:
comparision = pd.DataFrame()

In [68]:
comparision['MODEL NAME'] = ['OneVsRest','OneVsRest','OneVsRest','OneVsRest']

In [69]:
comparision['TEXT EXTRACTION METHOD'] = ['TF_IDF','TF_IDF_HYPERPARAMETER','BAG OF WORDS(UNIGRAM)','BAG OF WORDS(BIGRAM)']

In [70]:
comparision['ACCURACY'] = [TF_IDF_ACC,TFIDF_HYPER_ACC,BOW_UNI_ACC,BOW_BI_ACC]

In [71]:
comparision['PRECESSION'] = [TF_IDF_PREC,TFIDF_HYPER_PREC,BOW_UNI_PREC,BOW_BI_PREC]

In [72]:
comparision['RECALL'] = [TF_IDF_RE,TFIDF_HYPER_RE,BOW_UNI_RE,BOW_BI_RE]

In [73]:
comparision['F1_SCORE'] = [TF_IDF_F1,TFIDF_HYPER_F1,BOW_UNI_F1,BOW_BI_F1]

In [74]:
comparision

,MODEL NAME,TEXT EXTRACTION METHOD,ACCURACY,PRECESSION,RECALL,F1_SCORE
0,OneVsRest,TF_IDF,87.94,84.58,87.94,84.78
1,OneVsRest,TF_IDF_HYPERPARAMETER,88.00,85.00,88.00,85.00
2,OneVsRest,BAG OF WORDS(UNIGRAM),87.00,84.00,87.00,84.00
3,OneVsRest,BAG OF WORDS(BIGRAM),87.00,84.00,87.00,84.00


# ONE VS REST ON  PATIO 

In [75]:
# Load Data
df_patio =pd.read_csv('PATIO_COMPLETELY_CLEAN_FULLY_MERGED_RDY_FOR_MDLLING')

In [76]:
df_patio.shape

(932665, 10)

In [77]:
df_patio.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE
0,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,AUVPE2KEXBJVT,5.0,True,I don't spend a lot on my flags because they r...,2011-12-11
1,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A2F6GES1MBOFXS,5.0,True,"Super fast processing and shipping, if you are...",2011-07-30
2,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1SDVD3SZI1BAK,5.0,True,Great product. I would recommend this product...,2016-05-04
3,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1ZQJ3KCSLUPR3,5.0,True,GREAT PRICE I LOVE MY STATE AND COUNTRY,2015-11-02
4,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,ANU8FBZM618M3,5.0,True,Great display flag for the den.,2015-09-01


In [78]:
df_patio.drop_duplicates(subset=None,keep='first',inplace=True)

In [79]:
df_patio.shape

(727480, 10)

In [80]:
sentiment_patio=[]
for i in df_patio['rating']:
    if (i>3):
        sentiment_patio.append('Positive')
    elif(i<3):
        sentiment_patio.append('Negative')
    else:
        sentiment_patio.append('Neutral')

In [81]:
df_patio['ReviewSentiment']=sentiment_patio

In [82]:
df_patio.shape

(727480, 11)

In [83]:
df_patio.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE,ReviewSentiment
0,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,AUVPE2KEXBJVT,5.0,True,I don't spend a lot on my flags because they r...,2011-12-11,Positive
1,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A2F6GES1MBOFXS,5.0,True,"Super fast processing and shipping, if you are...",2011-07-30,Positive
2,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1SDVD3SZI1BAK,5.0,True,Great product. I would recommend this product...,2016-05-04,Positive
3,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1ZQJ3KCSLUPR3,5.0,True,GREAT PRICE I LOVE MY STATE AND COUNTRY,2015-11-02,Positive
4,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,ANU8FBZM618M3,5.0,True,Great display flag for the den.,2015-09-01,Positive


In [84]:
Text_patio  = df_patio[["reviewText","ReviewSentiment"]]

In [85]:
Text_patio.head()

,reviewText,ReviewSentiment
0,I don't spend a lot on my flags because they r...,Positive
1,"Super fast processing and shipping, if you are...",Positive
2,Great product. I would recommend this product...,Positive
3,GREAT PRICE I LOVE MY STATE AND COUNTRY,Positive
4,Great display flag for the den.,Positive


# TEXT PREPROCESSING

In [86]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

print(len(stopwords))
#print(stopwords)

326


In [87]:
def clean_text(text):
    # Split the text into individual words
    words = text.split()
    
    # Remove any punctuation or non-alphabetic characters
    words = [word for word in words if word.isalpha()]
    
    # Convert all words to lowercase
    words = [word.lower() for word in words]
    
    # Remove any stop words
    stop_words = stopwords=('english')
    words = [word for word in words if word not in stop_words]
    
    # Join the words back into a single string
    return " ".join(words)


# Clean the "text" column using the clean_text() function
Text_patio["reviewText"] = Text_patio["reviewText"].apply(clean_text)

# Print the cleaned data
print(Text_patio)

                                               reviewText ReviewSentiment
0       spend a lot on my flags because they really ge...        Positive
1       super fast processing and if you are looking f...        Positive
2         great would recommend this would recommend this        Positive
3                   great price love my state and country        Positive
4                              great display flag for the        Positive
...                                                   ...             ...
932660                    perfect for the project at hand        Positive
932661                                               like        Positive
932662       just what needed to use in my just the right        Positive
932663  perfect for what needed it my bonsais humidity...        Positive
932664  very small no more than a cup of stones cheape...        Negative

[727480 rows x 2 columns]


In [88]:
Text_patio["reviewText"]=Text_patio["reviewText"].apply(lambda x:' '.join([word for word in x.split() if word not in stopwords]))

# CHECKING FOR FIRST 100 RECORDS 

In [89]:
c = 0 
for i in Text_patio["reviewText"]:
    print(i)
    c+=1
    if c==100:
        break

spend lot flags beat lesser quality figuring replaced couple times year fact inexpensive flag flying months later pleases sees winds averaging mph day occasional storm friont gusts reaching high far months starting little fraying buying shortly ready finally gives
super fast processing looking flags look good quality product great company business
great recommend recommend
great price love state country
great display flag
cheap polyester flag serves purpose school color vibrant size true x
serves purpose holding little concerned fading think good
great quality looks
loved
knee saver taking vynal tile basement
tool worked remove partial old hardwood works nicely driveway ice frozen snow chipper remover flooring task fiberglass handle ability bend actually assisted removal process unbendable metal need additional buy
gave sharp edge strong
remove old linoleum job lot
bit big use worked got old vinyl great help removing carpet pad stuck
heavy heavy good tool hand
good
tool saved purchased

In [90]:
print(stopwords)

{'side', 'me', 'all', 'when', '‘re', 'their', 'becomes', 'an', 'many', 'n‘t', 'anyway', 'unless', 'indeed', 'we', 'yet', "'re", 'for', '’d', 'they', 'or', 'several', 'hereupon', 'though', 'ca', 'every', 'up', 'again', 'get', 'whereby', 'nevertheless', 'everywhere', 'nowhere', 'therein', 'enough', 'such', 'just', 'my', 'third', "'ve", 'has', 'with', 'off', 'beforehand', 'its', 'towards', 'take', 'always', 'whence', 'are', 'anything', 'nine', 'really', 'against', 'last', 'latter', 'seeming', 'toward', 'hereby', 'two', 'why', 'more', '‘ll', 'something', 'everything', 'himself', 'on', 'most', 'along', 'and', 'does', 'down', 'whenever', 'done', 'otherwise', 'thereupon', 'during', 'also', 'throughout', 'meanwhile', '‘m', 'how', 'namely', 'yourself', 'see', 'therefore', 'another', 'perhaps', 'either', 'ten', 'below', 'both', 'next', 'hundred', 'serious', 'doing', 'others', 'upon', 'have', 'what', 'name', 'never', '‘ve', 'became', 'quite', 'keep', "n't", 'go', 'give', 'your', 'make', 'herein',

In [91]:
Text_patio.head()

,reviewText,ReviewSentiment
0,spend lot flags beat lesser quality figuring r...,Positive
1,super fast processing looking flags look good ...,Positive
2,great recommend recommend,Positive
3,great price love state country,Positive
4,great display flag,Positive


In [92]:
# train test split 

In [93]:
X_patio=[]
y_patio=[]
for i in Text_patio['reviewText']:
    X_patio.append(i)
for j in Text_patio['ReviewSentiment']:
    y_patio.append(j)  

In [94]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train_patio,X_test_patio,y_train_patio,y_test_patio=train_test_split(X_patio,y_patio,test_size=0.20,random_state=9)

In [95]:
len(X_train_patio)

581984

In [96]:
len(X_test_patio)

145496

### convert text into vector by using tfidf vectorizer 

In [97]:
tfidf_vectorizer = TfidfVectorizer()
X_train_patio = tfidf_vectorizer.fit_transform(X_train_patio)
X_test_patio = tfidf_vectorizer.transform(X_test_patio)

In [98]:
print("X_train_patio",X_train_patio.shape)
print("X_test_patio",X_test_patio.shape)


X_train_patio (581984, 71855)
X_test_patio (145496, 71855)


## one vs rest strategy with logistic regression 

In [99]:
lr = LogisticRegression(max_iter=200)
ovr = OneVsRestClassifier(lr)

In [100]:
# fit the model on the training data
ovr.fit(X_train_patio, y_train_patio)
y_pred_patio = ovr.predict(X_test_patio)

In [101]:
df_compare_patio=pd.DataFrame()
df_compare_patio['Actual']=y_test_patio
df_compare_patio['Predicted']=y_pred_patio

In [102]:
df_compare_patio

,Actual,Predicted
0,Positive,Positive
1,Positive,Positive
2,Positive,Positive
3,Negative,Positive
4,Positive,Positive
...,...,...
145491,Positive,Positive
145492,Positive,Positive
145493,Positive,Positive
145494,Positive,Positive


In [103]:
print(classification_report(y_test_patio, y_pred_patio))

              precision    recall  f1-score   support

    Negative       0.68      0.46      0.55     14985
     Neutral       0.42      0.04      0.07     10452
    Positive       0.88      0.98      0.93    120059

    accuracy                           0.86    145496
   macro avg       0.66      0.49      0.52    145496
weighted avg       0.82      0.86      0.83    145496



In [104]:
confusion_matrix(y_test_patio,y_pred_patio)

array([[  6922,    200,   7863],
       [  1450,    421,   8581],
       [  1769,    391, 117899]], dtype=int64)

In [105]:
print('accuracy:', accuracy_score(y_test_patio, y_pred_patio))
print('recall:', recall_score(y_test_patio, y_pred_patio, average='weighted'))
print('f1-score:', f1_score(y_test_patio, y_pred_patio, average='weighted'))
print('precision:', precision_score(y_test_patio, y_pred_patio,average='weighted'))

accuracy: 0.8607934238741959
recall: 0.8607934238741959
f1-score: 0.8268504630566935
precision: 0.8243519900790063


In [106]:
P_TF_IDF_ACC = round(100*accuracy_score(y_test_patio, y_pred_patio),2)
P_TF_IDF_PREC =  round(100*precision_score(y_test_patio, y_pred_patio,average='weighted'),2)
P_TF_IDF_RE=  round(100*recall_score(y_test_patio, y_pred_patio, average='weighted'),2)
P_TF_IDF_F1 =  round(100*f1_score(y_test_patio, y_pred_patio, average='weighted'),2)

# tfidf with hyperparameter

In [107]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train_patio,X_test_patio,y_train_patio,y_test_patio=train_test_split(X_patio,y_patio,test_size=0.20,random_state=9)

In [108]:

tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True,
                          max_df=0.5,
                          min_df=1,
                          use_idf=True,smooth_idf=True)
X_train_patio = tfidf_vectorizer.fit_transform(X_train_patio)
X_test_patio = tfidf_vectorizer.transform(X_test_patio)




In [109]:
print("X_train_patio",X_train_patio.shape)
print("X_test_patio",X_test_patio.shape)


X_train_patio (581984, 71855)
X_test_patio (145496, 71855)


In [110]:
lr = LogisticRegression(max_iter=200)
ovr = OneVsRestClassifier(lr)

In [111]:
# fit the model on the training data
ovr.fit(X_train_patio, y_train_patio)
y_pred_patio = ovr.predict(X_test_patio)

In [112]:
df_compare_patio=pd.DataFrame()
df_compare_patio['Actual']=y_test_patio
df_compare_patio['Predicted']=y_pred_patio

In [113]:
df_compare_patio

,Actual,Predicted
0,Positive,Positive
1,Positive,Positive
2,Positive,Positive
3,Negative,Positive
4,Positive,Positive
...,...,...
145491,Positive,Positive
145492,Positive,Positive
145493,Positive,Positive
145494,Positive,Positive


In [114]:
print(classification_report(y_test_patio, y_pred_patio))

              precision    recall  f1-score   support

    Negative       0.68      0.47      0.55     14985
     Neutral       0.42      0.04      0.08     10452
    Positive       0.88      0.98      0.93    120059

    accuracy                           0.86    145496
   macro avg       0.66      0.50      0.52    145496
weighted avg       0.83      0.86      0.83    145496



In [115]:
confusion_matrix(y_test_patio,y_pred_patio)

array([[  6993,    203,   7789],
       [  1459,    434,   8559],
       [  1789,    391, 117879]], dtype=int64)

In [116]:
print('accuracy:', accuracy_score(y_test_patio, y_pred_patio))
print('recall:', recall_score(y_test_patio, y_pred_patio, average='weighted'))
print('f1-score:', f1_score(y_test_patio, y_pred_patio, average='weighted'))
print('precision:', precision_score(y_test_patio, y_pred_patio,average='weighted'))

accuracy: 0.8612332985099247
recall: 0.8612332985099247
f1-score: 0.8275797044121755
precision: 0.8253257618545138


In [117]:
P_TFIDF_HYPER_ACC =  round(accuracy_score(y_test_patio, y_pred_patio),2)*100
P_TFIDF_HYPER_PREC = round(precision_score(y_test_patio, y_pred_patio,average='weighted'),2)*100
P_TFIDF_HYPER_RE = round(recall_score(y_test_patio, y_pred_patio, average='weighted'),2)*100
P_TFIDF_HYPER_F1 = round(f1_score(y_test_patio, y_pred_patio, average='weighted'),2)*100

# bag of words (patio) 

In [118]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train_patio,X_test_patio,y_train_patio,y_test_patio=train_test_split(X_patio,y_patio,test_size=0.20,random_state=9)

# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer()
X_train_patio=cv.fit_transform(X_train_patio)


print(X_train_patio.shape)



X_test_patio=cv.transform(X_test_patio)

print(X_test_patio.shape)


print("X_train",X_train_patio.shape)
print("X_test",X_test_patio.shape)


(581984, 71855)
(145496, 71855)
X_train (581984, 71855)
X_test (145496, 71855)


In [119]:
# fit the model on the training data
ovr.fit(X_train_patio, y_train_patio)
y_pred = ovr.predict(X_test_patio)

df_compare_patio=pd.DataFrame()
df_compare_patio['Actual']=y_test_patio
df_compare_patio['Predicted']=y_pred_patio


df_compare_patio

print(classification_report(y_test_patio, y_pred_patio))

confusion_matrix(y_test_patio,y_pred_patio)

print('accuracy:', accuracy_score(y_test_patio, y_pred_patio))
print('recall:', recall_score(y_test_patio, y_pred_patio, average='weighted'))
print('f1-score:', f1_score(y_test_patio, y_pred_patio, average='weighted'))
print('precision:', precision_score(y_test_patio, y_pred_patio,average='weighted'))

P_BOW_UNI_ACC =  round(accuracy_score(y_test_patio, y_pred_patio),2)*100
P_BOW_UNI_PREC = round(precision_score(y_test_patio, y_pred_patio,average='weighted'),2)*100
P_BOW_UNI_RE = round(recall_score(y_test_patio, y_pred_patio, average='weighted'),2)*100
P_BOW_UNI_F1 = round(f1_score(y_test_patio, y_pred_patio, average='weighted'),2)*100

              precision    recall  f1-score   support

    Negative       0.68      0.47      0.55     14985
     Neutral       0.42      0.04      0.08     10452
    Positive       0.88      0.98      0.93    120059

    accuracy                           0.86    145496
   macro avg       0.66      0.50      0.52    145496
weighted avg       0.83      0.86      0.83    145496

accuracy: 0.8612332985099247
recall: 0.8612332985099247
f1-score: 0.8275797044121755
precision: 0.8253257618545138


# N_gram ( patio) 

In [121]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train_patio,X_test_patio,y_train_patio,y_test_patio=train_test_split(X_patio,y_patio,test_size=0.20,random_state=9)

# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer(ngram_range=(2,2))
X_train_patio=cv.fit_transform(X_train_patio)


print(X_train_patio.shape)



X_test_patio=cv.transform(X_test_patio)

print(X_test_patio.shape)


print("X_train",X_train_patio.shape)
print("X_test",X_test_patio.shape)


(581984, 3246637)
(145496, 3246637)
X_train (581984, 3246637)
X_test (145496, 3246637)


In [122]:
# fit the model on the training data
ovr.fit(X_train_patio, y_train_patio)
y_pred = ovr.predict(X_test_patio)

df_compare_patio=pd.DataFrame()
df_compare_patio['Actual']=y_test_patio
df_compare_patio['Predicted']=y_pred_patio


df_compare_patio

print(classification_report(y_test_patio, y_pred_patio))

confusion_matrix(y_test_patio,y_pred_patio)

print('accuracy:', accuracy_score(y_test_patio, y_pred_patio))
print('recall:', recall_score(y_test_patio, y_pred_patio, average='weighted'))
print('f1-score:', f1_score(y_test_patio, y_pred_patio, average='weighted'))
print('precision:', precision_score(y_test_patio, y_pred_patio,average='weighted'))

P_BOW_BI_ACC =  round(accuracy_score(y_test_patio, y_pred_patio),2)*100
P_BOW_BI_PREC = round(precision_score(y_test_patio, y_pred_patio,average='weighted'),2)*100
P_BOW_BI_RE = round(recall_score(y_test_patio, y_pred_patio, average='weighted'),2)*100
P_BOW_BI_F1 = round(f1_score(y_test_patio, y_pred_patio, average='weighted'),2)*100

              precision    recall  f1-score   support

    Negative       0.68      0.47      0.55     14985
     Neutral       0.42      0.04      0.08     10452
    Positive       0.88      0.98      0.93    120059

    accuracy                           0.86    145496
   macro avg       0.66      0.50      0.52    145496
weighted avg       0.83      0.86      0.83    145496

accuracy: 0.8612332985099247
recall: 0.8612332985099247
f1-score: 0.8275797044121755
precision: 0.8253257618545138


In [123]:
P_comparision = pd.DataFrame()
P_comparision['MODEL NAME'] = ['OneVsRest','OneVsRest','OneVsRest','OneVsRest']
P_comparision['TEXT EXTRACTION METHOD'] = ['TF_IDF','TF_IDF_HYPERPARAMETER','BAG OF WORDS(UNIGRAM)','BAG OF WORDS(BIGRAM)']
P_comparision['ACCURACY'] = [P_TF_IDF_ACC,P_TFIDF_HYPER_ACC,P_BOW_UNI_ACC,P_BOW_BI_ACC]
P_comparision['PRECESSION'] = [P_TF_IDF_PREC,P_TFIDF_HYPER_PREC,P_BOW_UNI_PREC,P_BOW_BI_PREC]
P_comparision['RECALL'] = [P_TF_IDF_RE,P_TFIDF_HYPER_RE,P_BOW_UNI_RE,P_BOW_BI_RE]
P_comparision['F1_SCORE'] = [P_TF_IDF_F1,P_TFIDF_HYPER_F1,P_BOW_UNI_F1,P_BOW_BI_F1]
P_comparision

,MODEL NAME,TEXT EXTRACTION METHOD,ACCURACY,PRECESSION,RECALL,F1_SCORE
0,OneVsRest,TF_IDF,86.08,82.44,86.08,82.69
1,OneVsRest,TF_IDF_HYPERPARAMETER,86.00,83.00,86.00,83.00
2,OneVsRest,BAG OF WORDS(UNIGRAM),86.00,83.00,86.00,83.00
3,OneVsRest,BAG OF WORDS(BIGRAM),86.00,83.00,86.00,83.00
